In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *
from sklearn.metrics import roc_auc_score
from torchvision.models import *

In [6]:
path = Path('.')
!ls {path}'./input/'

densenet169  models		    subm_20.csv   test
dn269	     sample_submission.csv  _subm_30.csv  tmp
labels.csv   subm		    _subm_41.csv  train


In [ ]:
tfms = get_transforms(do_flip=True, flip_vert=True, max_zoom=.1,
                      max_lighting=0.05, max_warp=0.)
#tfms = get_transforms(xtra_tfms=cutout())
data = ImageDataBunch.from_csv(path, folder='train', ds_tfms=tfms, size=96, test='test', suffix='.tif', bs=128)
#ds_tfms=tfms, 
data.normalize(imagenet_stats)
data.show_batch(rows=3, figsize=(6,6))

In [ ]:
def auc_score(y_pred,y_true,tens=True):
    score=roc_auc_score(y_true,torch.sigmoid(y_pred)[:,1])
    if tens:
        score=tensor(score)
    else:
        score=score
    return score

In [ ]:
#learn = create_cnn(data, models.resnet34, metrics=[accuracy, auc_score])

In [ ]:
#learn.fit_one_cycle(2, max_lr=.02)

In [ ]:
#learn = create_cnn(data, models.resnet50, metrics=[accuracy, auc_score])

In [ ]:
#learn.fit_one_cycle(2, max_lr=.02)

In [ ]:
#learn = create_cnn(data, models.resnet152, metrics=[accuracy, auc_score])

In [ ]:
#learn.fit_one_cycle(2, max_lr=.02)

In [ ]:
#learn = create_cnn(data, densenet169, ps=0.5, metrics=[accuracy, auc_score])

In [ ]:
learn = create_cnn(data, vgg16, ps=0.5, metrics=[accuracy, auc_score])

In [ ]:
learn.fit_one_cycle(2, max_lr=.02)

In [ ]:
lr_find(learn)

learn.recorder.plot()

In [ ]:
learn.precompute=False
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(3, max_lr=slice(1e-5, 1e-3))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(16)

In [ ]:
preds_t,_ = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
learn.export('densenet169')

In [ ]:
def submit(name, sample_file, learn, preds_t):
    sample_ds = pd.read_csv(sample_file).set_index('id')
    clean_fname=np.vectorize(lambda fname: str(fname).split('/')[-1].split('.')[0])
    fname=clean_fname(learn.data.test_ds.items) #(data.test_ds.items) learn.data.test_ds.fnames
    fname=fname.astype(str)
    sample_ds.loc[fname,'label']=to_np(preds_t[:,1])
    sample_ds.to_csv(name)
    !kaggle competitions submit -c histopathologic-cancer-detection -f {name} -m "{name}"

In [ ]:
submit('HCD_1.csv', path/'sample_submission.csv', learn, preds_t)

## The END